# SISTEMAS DE RECOMENDACIÓN

Se carga el dataset Adult Income desde una fuente oficial y se visualizan sus primeras filas para conocer su estructura y preparar el análisis.

In [15]:
import pandas as pd

total_data = pd.read_csv("https://breathecode.herokuapp.com/asset/internal-link?id=2326&path=adult-census-income.csv")


print(total_data.shape)
print(total_data.head())

(32561, 15)
   age workclass  fnlwgt     education  education.num marital.status  \
0   90         ?   77053       HS-grad              9        Widowed   
1   82   Private  132870       HS-grad              9        Widowed   
2   66         ?  186061  Some-college             10        Widowed   
3   54   Private  140359       7th-8th              4       Divorced   
4   41   Private  264663  Some-college             10      Separated   

          occupation   relationship   race     sex  capital.gain  \
0                  ?  Not-in-family  White  Female             0   
1    Exec-managerial  Not-in-family  White  Female             0   
2                  ?      Unmarried  Black  Female             0   
3  Machine-op-inspct      Unmarried  White  Female             0   
4     Prof-specialty      Own-child  White  Female             0   

   capital.loss  hours.per.week native.country income  
0          4356              40  United-States  <=50K  
1          4356              18  U

Se limpian los datos eliminando valores faltantes, se convierten las variables categóricas en numéricas mediante one-hot encoding y se normalizan las variables numéricas para dejar el dataset listo para el modelado.

In [16]:
# Reemplazar valores "?" por NaN
total_data = total_data.replace('?', pd.NA)

# Eliminar filas con valores faltantes
total_data = total_data.dropna()

# Quitar espacios en los nombres de columnas
total_data.columns = total_data.columns.str.strip()

# Variables categóricas a tipo "category"
cat_cols = total_data.select_dtypes(include='object').columns
num_cols = total_data.select_dtypes(include=['int64', 'float64']).columns

# Codificar variables categóricas
total_data_encoded = pd.get_dummies(total_data, columns=cat_cols, drop_first=True)

# Normalizar las numéricas
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
total_data_encoded[num_cols] = scaler.fit_transform(total_data_encoded[num_cols])

total_data_encoded.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,income_>50K
1,3.316630,-0.538790,-0.439738,-0.147445,10.555814,-1.914161,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,1.184831,-0.467906,-2.400559,-0.147445,9.427915,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0.195067,0.708645,-0.047574,-0.147445,9.427915,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
5,-0.337883,0.256222,-0.439738,-0.147445,9.106365,0.339636,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
6,-0.033340,-0.370964,-1.616231,-0.147445,9.106365,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False


Se entrena un modelo KNN con los perfiles de personas que ganan más de 50 K para identificar usuarios similares de alto ingreso y así recomendar a los de menor ingreso qué características podrían mejorar.

In [17]:
from sklearn.neighbors import NearestNeighbors

# Dividir entre los que ganan más de 50K y los que no
total_data_high = total_data_encoded[total_data['income'] == '>50K']
total_data_low = total_data_encoded[total_data['income'] == '<=50K']

# Modelo KNN para perfiles de alto ingreso
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(total_data_high)

# Seleccionamos un usuario hipotético de bajo ingreso
sample_user = total_data_low.sample(1, random_state=42)
distances, indices = knn.kneighbors(sample_user, n_neighbors=5)

# Mostrar usuarios similares de alto ingreso
recommendations = total_data_high.iloc[indices[0]]
recommendations.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,income_>50K
25442,-0.794697,-1.498334,1.128918,-0.147445,-0.218586,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
19260,-0.794697,-1.372694,1.128918,-0.147445,-0.218586,0.757005,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
17421,-0.337883,-0.521119,1.128918,-0.147445,-0.218586,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
27288,-0.946968,-0.123035,1.128918,-0.147445,-0.218586,-0.077734,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
7009,-0.337883,-1.537264,1.128918,-0.147445,-0.218586,0.757005,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True


Se entrena un modelo Random Forest para predecir la probabilidad de que una persona gane más de 50 K. Luego se simula un cambio en su nivel educativo para observar cómo aumentaría su ingreso estimado.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = total_data_encoded.drop(columns=['income_>50K'])
y = total_data_encoded['income_>50K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Probabilidad actual del usuario
sample_user = X_test.sample(1, random_state=42)
prob_actual = model.predict_proba(sample_user)[0][1]

# Simular que estudia más (cambia nivel educativo)
sample_sim = sample_user.copy()
sample_sim['education.num'] += 2
prob_new = model.predict_proba(sample_sim)[0][1]

print(f"Probabilidad actual de >50K: {prob_actual:.2%}")
print(f"Probabilidad si estudia más: {prob_new:.2%}")

Probabilidad actual de >50K: 70.00%
Probabilidad si estudia más: 70.50%


Se crean perfiles simulados de usuarios para probar el modelo y analizar cómo diferentes características (edad, educación, ocupación, horas trabajadas) afectan la probabilidad de tener ingresos altos.

In [19]:
profiles = [
    {"age": 25, "education.num": 10, "hours-per-week": 35, "occupation": "Sales"},
    {"age": 40, "education.num": 13, "hours-per-week": 50, "occupation": "Exec-managerial"},
]